In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from io import BytesIO
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Initialize variables
E_voltages = None
input_widgets = None

# Create an Output widget for displaying dynamic content
output_box = widgets.Output()

# Function to create and display the input grid using ipywidgets
def create_input_grid(rows, cols, prefilled_data):
    global E_voltages, input_widgets

    # Initialize empty array for E_voltages
    E_voltages = np.zeros((rows, cols))

    # If prefilled_data is provided, update charges with it
    if prefilled_data is not None and prefilled_data.shape == (rows, cols):
        E_voltages = prefilled_data
        
    # Create a list to hold the FloatText widgets
    input_widgets = []

    # Clear the output box before displaying new content
    with output_box:
        clear_output(wait=True)
        
    # Create FloatText widgets and store them in input_widgets
    for i in range(rows):
        row_widgets = []
        for j in range(cols):
            widget = widgets.FloatText(value=E_voltages[i, j], description=f'[{rows - 1 - i}, {j}]')
            input_widgets.append(widget)
            row_widgets.append(widget)
        display(widgets.HBox(row_widgets))

# Function to handle widget value changes and update E_voltages
def update_E_voltages(change):
    global E_voltages, input_widgets
    rows, cols = E_voltages.shape
    for i in range(rows):
        for j in range(cols):
            E_voltages[i, j] = input_widgets[(rows - 1 - i) * cols + j].value  # Reverse row index

# Function to handle file upload and process the Excel file
def on_file_upload(change):
    global E_voltages, input_widgets
    file_contents = excel_upload_button.value[0]['content']
    excel_file = BytesIO(file_contents)
    df = pd.read_excel(excel_file, header=None)  # Read without assuming headers

    # Convert to numeric and drop empty rows and columns
    df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all').dropna(axis=1, how='all')

    # Ensure to handle any non-numeric headers or rows that were meant to be included
    if df.iloc[0].isnull().all():  # Check if the first row is entirely empty and drop if so
        df = df.drop(0)

    # Replace empty values with 0
    df = df.fillna(0)

    # Find non-empty indices to determine the relevant data range
    non_empty_indices = np.argwhere(~np.isnan(df.to_numpy()))
    start_row, start_col = non_empty_indices.min(axis=0)
    end_row, end_col = non_empty_indices.max(axis=0)
    E_voltages = df.iloc[start_row:end_row + 1, start_col:end_col + 1].to_numpy()

    # Clear previous input widgets and display updated grid
    with output_box:
        clear_output(wait=True)
        create_input_grid(E_voltages.shape[0], E_voltages.shape[1], prefilled_data=E_voltages)

# Function to update plots
def update_plots(b=None):
    global E_voltages

    if E_voltages is None:
        return  # Return if E_voltages is not initialized

    # Calculate the max of each 2x2 sub-array
    E_max = np.array([
            [E_voltages[i:i+2, j:j+2].max() for j in range(E_voltages.shape[1]-1)]
            for i in range(E_voltages.shape[0]-1)
        ])

    # Variables for the change in x and y
    delta_x = 50
    delta_y = 50

    # Calculate Ex and Ey components
    Ex = -1 * np.diff(E_voltages, axis=1) / delta_x
    Ey = -1 * np.diff(E_voltages, axis=0) / delta_y

    # Remove the Ex extra row and Ey extra column
    Ex = Ex[:-1:]
    Ey = Ey[:, :-1]

    # Calculate the magnitude based on Ex and Ey components
    E_magnitude = np.sqrt(Ex**2 + Ey**2)

    # Add a small epsilon value to avoid division by zero
    epsilon = 1e-10  # A very small value
    E_magnitude += epsilon

    # Calculate the unit vector of the Ex and Ey (Ex or Ey / E_magnitude)
    Ex_normalized = Ex / E_magnitude
    Ey_normalized = Ey / E_magnitude

    # Set the meshgrid for the plots
    x = np.arange(Ex.shape[1])
    y = np.arange(Ex.shape[0])
    X, Y = np.meshgrid(x, y)

    # Clear previous plots within the output box
    with output_box:
        clear_output(wait=True)

        # Set up figure and axes
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        fig.suptitle('Electric Field', fontsize=40)

        # First subplot (Unit Vector Field)
        axs[0].quiver(X, Y, Ex_normalized, Ey_normalized, color='b')
        axs[0].set_title('UNIT VECTOR', fontsize=17)
        axs[0].set_xlabel('x')
        axs[0].set_ylabel('y')

        # Adjust the plot limits for complete visibility
        axs[0].set_xlim(-1, X.max() + 1)
        axs[0].set_ylim(-1, Y.max() + 1)
        axs[0].grid(True)

        # Second subplot (Stream Field)
        stream = axs[1].streamplot(X, Y, Ex, Ey, density=0.5, linewidth=1.5, arrowsize=2, color=E_max, cmap='gist_heat_r', broken_streamlines=False)
        colorbar = plt.colorbar(stream.lines, ax=axs[1], shrink=.7)
        colorbar.set_label('Voltage', fontsize=12, weight='bold')
        axs[1].set_title('STREAM PLOT', fontsize=17)
        axs[1].set_xlabel('x')
        axs[1].set_ylabel('y')
        axs[1].grid(True)

        plt.tight_layout()
        plt.show()

# Create widgets for user input and file upload
info_heading = widgets.HTML(value="""
                            <h1>Miami Dade College | Hialeah Campus</h1>
                            <h2>Electric Field Plotter</h2>
                            """)
info_body = widgets.HTML(value="""
                        <p>This tool allows you to visualize electric fields based on a table of voltages.</p>
                        <p>Upload an Excel table with voltage values.</p>
                        """)

excel_upload_button = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload Excel Table")
update_button = widgets.Button(description="Run")

# Function to handle update button click event
def on_generate_plot(b):
    update_plots()

# Attach button click events
excel_upload_button.observe(on_file_upload, names='value')
update_button.on_click(on_generate_plot)

# Display initial widgets including the output box
display(info_heading, info_body, widgets.HBox([excel_upload_button, update_button]), output_box)